![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Simple Moving Average Crossover Strategy Analysis (Minute Resolution)

This notebook analyzes the SMA crossover strategy using minute-level data with 390-minute and 1170-minute simple moving averages on SPY.

**Strategy Configuration:**
- **Data Resolution**: MINUTE
- **Fast SMA**: 390 minutes (~1 trading day)
- **Slow SMA**: 1170 minutes (~3 trading days)
- **Buy Signal**: When fast SMA crosses above slow SMA (bullish momentum)
- **Sell Signal**: When fast SMA crosses below slow SMA (bearish momentum)
- **Position Size**: 95% of available capital
- **Backtest Period**: October 7-11, 2013 (4 trading days)

## Strategy Overview

In [ ]:
# Initialize QuantBook for analysis
qb = QuantBook()

# Add SPY equity with MINUTE resolution
spy = qb.add_equity("SPY", Resolution.MINUTE)

# Set analysis period (same as original main.py)
qb.set_start_date(2013, 10, 7)
qb.set_end_date(2013, 10, 11)

# Get minute-level data for SPY (same period as algorithm)
history = qb.history(qb.securities.keys(), Resolution.MINUTE)

print(f"Data range: {history.index[0]} to {history.index[-1]}")
print(f"Total data points: {len(history)}")
print(f"Starting price: ${history['close'].iloc[0]:.2f}")
print(f"Ending price: ${history['close'].iloc[-1]:.2f}")
print(f"Price change: {((history['close'].iloc[-1] / history['close'].iloc[0]) - 1) * 100:.2f}%")
print(f"High price: ${history['high'].max():.2f}")
print(f"Low price: ${history['low'].min():.2f}")
print(f"Average volume: {history['volume'].mean():.0f}")

## Calculate Moving Averages (Minute Data)

In [ ]:
# Calculate Simple Moving Averages for minute data
fast_period = 390  # ~1 trading day of minute data
slow_period = 1170  # ~3 trading days of minute data

history['SMA_fast'] = history['close'].rolling(window=fast_period).mean()
history['SMA_slow'] = history['close'].rolling(window=slow_period).mean()

# Calculate crossover signals
history['SMA_fast_prev'] = history['SMA_fast'].shift(1)
history['SMA_slow_prev'] = history['SMA_slow'].shift(1)

# Identify crossover points
history['bullish_crossover'] = (
    (history['SMA_fast'] > history['SMA_slow']) &
    (history['SMA_fast_prev'] <= history['SMA_slow_prev'])
)

history['bearish_crossover'] = (
    (history['SMA_fast'] < history['SMA_slow']) &
    (history['SMA_fast_prev'] >= history['SMA_slow_prev'])
)

print("Moving averages calculated successfully!")
print(f"Fast SMA period: {fast_period} minutes (~{fast_period/390:.1f} trading days)")
print(f"Slow SMA period: {slow_period} minutes (~{slow_period/390:.1f} trading days)")
print(f"Number of bullish crossovers: {history['bullish_crossover'].sum()}")
print(f"Number of bearish crossovers: {history['bearish_crossover'].sum()}")

# Check if we have valid SMA data
valid_sma_data = history.dropna(subset=['SMA_fast', 'SMA_slow'])
print(f"Valid SMA data points: {len(valid_sma_data)} out of {len(history)}")

## Visualize the Strategy (Minute Data)

In [ ]:
# Create the SMA crossover plot for minute data
fig, ax = plt.subplots(figsize=(20, 10))

# Plot price and moving averages (only where SMA data is available)
valid_data = history.dropna(subset=['SMA_fast', 'SMA_slow'])

ax.plot(valid_data.index, valid_data['close'], label='SPY Close', color='black', linewidth=0.8, alpha=0.7)
ax.plot(valid_data.index, valid_data['SMA_fast'], label='390-Min SMA (Fast)', color='blue', linewidth=2)
ax.plot(valid_data.index, valid_data['SMA_slow'], label='1170-Min SMA (Slow)', color='red', linewidth=2)

# Mark crossover points
buy_signals = valid_data[valid_data['bullish_crossover']]
sell_signals = valid_data[valid_data['bearish_crossover']]

if len(buy_signals) > 0:
    ax.scatter(buy_signals.index, buy_signals['close'], 
              color='green', marker='^', s=80, label='Buy Signal', zorder=5)

if len(sell_signals) > 0:
    ax.scatter(sell_signals.index, sell_signals['close'], 
              color='red', marker='v', s=80, label='Sell Signal', zorder=5)

ax.set_title('SPY SMA Crossover Strategy - Minute Data (Oct 7-11, 2013)', fontsize=16, fontweight='bold')
ax.set_ylabel('Price ($)', fontsize=12)
ax.set_xlabel('Time', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Format x-axis to show dates nicely
fig.autofmt_xdate()

plt.tight_layout()
plt.show()

# Display crossover information
if len(buy_signals) > 0:
    print("\nBuy Signals:")
    print("="*60)
    for i, (timestamp, row) in enumerate(buy_signals.iterrows()):
        print(f"{i+1}. {timestamp}: Price=${row['close']:.2f}, Fast SMA=${row['SMA_fast']:.2f}, Slow SMA=${row['SMA_slow']:.2f}")

if len(sell_signals) > 0:
    print("\nSell Signals:")
    print("="*60)
    for i, (timestamp, row) in enumerate(sell_signals.iterrows()):
        print(f"{i+1}. {timestamp}: Price=${row['close']:.2f}, Fast SMA=${row['SMA_fast']:.2f}, Slow SMA=${row['SMA_slow']:.2f}")

if len(buy_signals) == 0 and len(sell_signals) == 0:
    print("\nNo crossover signals detected in the analysis period.")
    print("This could be due to:")
    print("- Short analysis period (only 4 trading days)")
print("- SMA periods being too long for the available data")

## Strategy Performance Analysis

In [ ]:
# Simulate the SMA crossover strategy on minute data
initial_capital = 100000
cash = initial_capital
shares = 0
portfolio_values = []
trades = []
position_changes = []

for timestamp, row in valid_data.iterrows():
    current_price = row['close']
    
    # Check for buy signal
    if row['bullish_crossover'] and shares == 0:
        shares = int(cash * 0.95 / current_price)  # Use 95% of available cash
        cash -= shares * current_price
        trade_info = {
            'timestamp': timestamp,
            'type': 'BUY',
            'price': current_price,
            'shares': shares,
            'value': shares * current_price,
            'fast_sma': row['SMA_fast'],
            'slow_sma': row['SMA_slow']
        }
        trades.append(trade_info)
        position_changes.append(f"BUY: {shares} shares @ ${current_price:.2f} at {timestamp}")
    
    # Check for sell signal
    elif row['bearish_crossover'] and shares > 0:
        cash += shares * current_price
        trade_info = {
            'timestamp': timestamp,
            'type': 'SELL',
            'price': current_price,
            'shares': shares,
            'value': shares * current_price,
            'fast_sma': row['SMA_fast'],
            'slow_sma': row['SMA_slow']
        }
        trades.append(trade_info)
        position_changes.append(f"SELL: {shares} shares @ ${current_price:.2f} at {timestamp}")
        shares = 0
    
    # Calculate portfolio value
    portfolio_value = cash + shares * current_price
    portfolio_values.append(portfolio_value)

# Add portfolio values to valid data
valid_data = valid_data.copy()
valid_data['portfolio_value'] = portfolio_values

# Calculate performance metrics
final_value = cash + shares * valid_data['close'].iloc[-1] if shares > 0 else cash
total_return = (final_value / initial_capital - 1) * 100
buy_hold_return = ((valid_data['close'].iloc[-1] / valid_data['close'].iloc[0]) - 1) * 100

print(f"SMA Crossover Strategy Results (Minute Data):")
print(f"{'='*50}")
print(f"Analysis Period: {valid_data.index[0].date()} to {valid_data.index[-1].date()}")
print(f"Data Points: {len(valid_data):,} minute bars")
print(f"Initial Capital: ${initial_capital:,.2f}")
print(f"Final Portfolio Value: ${final_value:,.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Buy & Hold Return: {buy_hold_return:.2f}%")
print(f"Strategy vs Buy & Hold: {total_return - buy_hold_return:+.2f}%")
print(f"Number of Trades: {len(trades)}")
if shares > 0:
    print(f"Final Position: LONG {shares} shares")
print(f"Final Cash: ${cash:.2f}")

## Trade Analysis

In [ ]:
# Create a DataFrame of trades
import pandas as pd

trades_df = pd.DataFrame(trades)

if len(trades_df) > 0:
    print("All Trades:")
    print("="*100)
    for i, trade in trades_df.iterrows():
        print(f"{trade['timestamp']} | {trade['type']:4} | {trade['shares']:6} shares @ ${trade['price']:7.2f} = ${trade['value']:10.2f} | Fast SMA: ${trade['fast_sma']:7.2f} | Slow SMA: ${trade['slow_sma']:7.2f}")
    
    # Calculate profit/loss for completed trades
    completed_trades = []
    for i in range(0, len(trades_df) - 1, 2):
        if i + 1 < len(trades_df):
            buy_trade = trades_df.iloc[i]
            sell_trade = trades_df.iloc[i + 1]
            profit_loss = (sell_trade['price'] - buy_trade['price']) * buy_trade['shares']
            completed_trades.append({
                'buy_date': buy_trade['timestamp'],
                'sell_date': sell_trade['timestamp'],
                'buy_price': buy_trade['price'],
                'sell_price': sell_trade['price'],
                'shares': buy_trade['shares'],
                'pnl': profit_loss,
                'return_pct': (sell_trade['price'] / buy_trade['price'] - 1) * 100
            })
    
    if completed_trades:
        print(f"\nCompleted Trade Analysis:")
        print(f"{'='*80}")
        print(f"Total completed trades: {len(completed_trades)}")
        
        winning_trades = [t for t in completed_trades if t['pnl'] > 0]
        losing_trades = [t for t in completed_trades if t['pnl'] < 0]
        
        print(f"Winning trades: {len(winning_trades)}")
        print(f"Losing trades: {len(losing_trades)}")
        print(f"Win rate: {len(winning_trades) / len(completed_trades) * 100:.1f}%")
        
        if winning_trades:
            avg_win = sum(t['pnl'] for t in winning_trades) / len(winning_trades)
            print(f"Average winning trade: ${avg_win:,.2f}")
        
        if losing_trades:
            avg_loss = sum(abs(t['pnl']) for t in losing_trades) / len(losing_trades)
            print(f"Average losing trade: ${avg_loss:,.2f}")
        
        total_pnl = sum(t['pnl'] for t in completed_trades)
        print(f"Total P&L from completed trades: ${total_pnl:,.2f}")
        
        # Show detailed completed trades
        print(f"\nDetailed Completed Trades:")
        print(f"{'#':<3} {'Buy Date':<20} {'Buy Price':<10} {'Sell Date':<20} {'Sell Price':<10} {'Shares':<7} {'P&L':<10} {'Return %':<8}")
        print("-" * 100)
        for i, trade in enumerate(completed_trades, 1):
            print(f"{i:<3} {trade['buy_date'].strftime('%Y-%m-%d %H:%M'):<20} ${trade['buy_price']:<9.2f} {trade['sell_date'].strftime('%Y-%m-%d %H:%M'):<20} ${trade['sell_price']:<9.2f} {trade['shares']:<7} ${trade['pnl']:<9.2f} {trade['return_pct']:<7.2f}%")
else:
    print("No trades were executed during the analysis period.")
    print("\nPosition Changes:")
    for change in position_changes:
        print(f"  {change}")

## Portfolio Performance Visualization

In [ ]:
# Compare strategy performance with buy & hold
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12))

# Plot 1: Portfolio Value Comparison
valid_data['buy_hold_value'] = initial_capital * (valid_data['close'] / valid_data['close'].iloc[0])

ax1.plot(valid_data.index, valid_data['buy_hold_value'], label='Buy & Hold', color='blue', linewidth=2, alpha=0.8)
ax1.plot(valid_data.index, valid_data['portfolio_value'], label='SMA Crossover', color='green', linewidth=2)

ax1.set_title('Portfolio Value Comparison: SMA Crossover vs Buy & Hold (Minute Data)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Price and SMA Visualization
ax2.plot(valid_data.index, valid_data['close'], label='SPY Close', color='black', linewidth=1, alpha=0.7)
ax2.plot(valid_data.index, valid_data['SMA_fast'], label='390-Min SMA', color='blue', linewidth=2)
ax2.plot(valid_data.index, valid_data['SMA_slow'], label='1170-Min SMA', color='red', linewidth=2)

# Mark buy/sell signals
if len(buy_signals) > 0:
    ax2.scatter(buy_signals.index, buy_signals['close'], color='green', marker='^', s=60, label='Buy', zorder=5)
if len(sell_signals) > 0:
    ax2.scatter(sell_signals.index, sell_signals['close'], color='red', marker='v', s=60, label='Sell', zorder=5)

ax2.set_title('SPY Price with SMA Crossover Signals (Minute Data)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Price ($)', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Format x-axis for both plots
fig.autofmt_xdate()

plt.tight_layout()
plt.show()

# Calculate some additional statistics
strategy_returns = valid_data['portfolio_value'].pct_change().dropna()
buy_hold_returns = valid_data['buy_hold_value'].pct_change().dropna()

if len(strategy_returns) > 0 and len(buy_hold_returns) > 0:
    strategy_vol = strategy_returns.std() * (252 * 390)**0.5  # Annualized volatility for minute data
    buy_hold_vol = buy_hold_returns.std() * (252 * 390)**0.5
    
    strategy_sharpe = (strategy_returns.mean() * 252 * 390) / (strategy_returns.std() * (252 * 390)**0.5) if strategy_returns.std() > 0 else 0
    buy_hold_sharpe = (buy_hold_returns.mean() * 252 * 390) / (buy_hold_returns.std() * (252 * 390)**0.5) if buy_hold_returns.std() > 0 else 0
    
    print(f"\nRisk Metrics (Minute Data):")
    print(f"{'='*40}")
    print(f"Strategy Volatility: {strategy_vol:.4f}")
    print(f"Buy & Hold Volatility: {buy_hold_vol:.4f}")
    print(f"Strategy Sharpe Ratio: {strategy_sharpe:.4f}")
    print(f"Buy & Hold Sharpe Ratio: {buy_hold_sharpe:.4f}")

## Strategy Summary

### Key Configuration:
- **Data Resolution**: MINUTE bars
- **Fast SMA**: 390 minutes (~1 trading day)
- **Slow SMA**: 1170 minutes (~3 trading days)
- **Backtest Period**: October 7-11, 2013 (4 trading days)
- **Total Data Points**: Minute-level OHLCV data

### Analysis Results:
The minute-resolution SMA crossover strategy provides high-frequency signal generation compared to daily resolution.
Key observations from this 4-day backtest period:

### Advantages of Minute Resolution:
- **More Signals**: Higher frequency of potential trading opportunities
- **Faster Response**: Quicker reaction to price movements
- **Granular Analysis**: Better understanding of intraday price behavior
- **Fine-tuned Timing**: More precise entry and exit points

### Challenges with Minute Data:
- **Noise**: More false signals due to intraday volatility
- **Transaction Costs**: Higher frequency increases trading costs
- **Complexity**: More data points require more computational resources
- **Overfitting Risk**: Shorter periods may not capture true market trends

### Important Considerations:
- **Short Backtest Period**: Only 4 trading days may not be representative
- **Market Conditions**: October 2013 was a specific market environment
- **SMA Periods**: 390 and 1170 minutes may be too long for short backtests
- **Transaction Costs**: Real trading would incur fees that reduce returns

### Recommendations:
1. **Extend Backtest Period**: Use at least 1-2 years of data for robust analysis
2. **Optimize Parameters**: Test different SMA combinations for minute data
3. **Add Filters**: Implement volume or volatility filters to reduce noise
4. **Include Costs**: Account for transaction fees and slippage
5. **Risk Management**: Add position sizing and stop-loss mechanisms
6. **Out-of-Sample Testing**: Validate strategy on different time periods

This minute-resolution analysis provides insights into high-frequency SMA crossover behavior but should be combined with longer-term studies for comprehensive strategy evaluation.